# Query News Crawler

## 0. imports

In [1]:
%load_ext lab_black

In [2]:
import sys

sys.path.append("..")

In [3]:
import os
import re
import time
import random
import dill
import requests
import urllib
import bs4

from collections import defaultdict
from datetime import datetime, timedelta
from typing import List, Dict, Union

from bs4 import BeautifulSoup
from bs4.element import NavigableString
from kss import split_sentences

from newspaper import Article
from tqdm import tqdm, trange
from crawler.utils import clean_text

## 1. QueryNewsCrawler class

### 1.1 line by line coding

In [28]:
query = "코스피"
query = urllib.parse.quote(query)

main_url = "https://search.naver.com/search.naver"
query_url = "?where=news&sm=tab_pge&query="
# 네이버 뉴스의 경우 page가 아닌
# 뉴스 기사 개수 카운팅으로 보여주고 있음
# 1 ~ 4000개까지 10개 단위로 보여줌
page_url = "&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=23&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start="

start = 1
start_range = list(range(1, 4000, 10))
url = f"{main_url}{query_url}{query}{page_url}{start}"

In [29]:
# req = requests.get(url)
# soup = BeautifulSoup(req.text, "html.parser")

# # ul lists
# ul_lists = soup.find("ul", {"class": "list_news"})
# # news links
# links = ul_lists.find_all("a", {"class": "news_tit"})
# link_title_dict = {row["href"]: {"title": row["title"]} for row in links}


# # update news texts
# news_texts = ul_lists.find_all("a", {"class": "api_txt_lines dsc_txt_wrap"})
# for row in news_texts:
#     link_title_dict[row["href"]]["text"] = row.text

In [72]:
# request and soup
req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")

# ul lists and li lists
ul_lists = soup.find("ul", {"class": "list_news"})
li_lists = ul_lists.findChildren("li", {"class": "bx"})

news_data = []
for li in li_lists:

    # news link & title
    link = li.find("a", {"class": "news_tit"})
    href, title = link["href"], link["title"]

    # news text
    text = li.find("a", {"class": "api_txt_lines dsc_txt_wrap"}).text

    # thumb nail if exist
    thumb = ""
    thumb_link = li.find("img", {"class": "thumb api_get"})
    if thumb_link:
        thumb = thumb_link["src"]

    # update link_title_dict
    news_data.append({"href": href, "title": title, "text": text, "thumb": thumb})

In [31]:
# li = li_lists[3]
# li.find_all("a", {"class": "info"})
# # li.find("span", {"class": "info"}).text

In [99]:
# url

In [101]:
li = li_lists[0]

In [102]:
# 언론사
press = li.find("a", {"class": "info press"}).text

In [103]:
# 뉴스 링크
links = li.find_all("a", {"class": "info"})
naver_news_url = [link for link in links if link.text == "네이버뉴스"]
if naver_news_url:
    naver_news_url = naver_news_url[0]
    naver_news_url = naver_news_url["href"].replace("&amp;", "&")
else:
    naver_news_url = None

In [104]:
naver_news_url

'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=015&aid=0004675733'

In [105]:
a = Article(naver_news_url, language="ko")
a.download()
a.parse()

In [ ]:
type(a)

In [109]:
soup = BeautifulSoup(a.html, "html.parser")

In [111]:
soup.find("span", {"class": "t11"})

<span class="t11">2022.03.20. 오전 8:02</span>

In [55]:
# links = li.find_all("a", {"class": "info"})
# naver_news_url = [link for link in links if link.text == "네이버뉴스"][0]
# naver_news_url = naver_news_url["href"].replace("&amp;", "&")

# # 언어가 한국어이므로 language='ko'로 설정
# a = Article(naver_news_url, language="ko")
# a.download()
# a.parse()

In [8]:
# for news in news_data:
#     href = news["href"]

#     # 언어가 한국어이므로 language='ko'로 설정
#     a = Article(href, language="ko")
#     a.download()
#     a.parse()

#     news["date"] = a.publish_date

In [10]:
# news_data

### 1.2 Class

In [66]:
class QueryNewsCrawler:
    def __init__(self):
        # 네이버 뉴스의 경우 page가 아닌
        # 뉴스 기사 개수 카운팅으로 보여주고 있음
        # 1 ~ 4000개까지 10개 단위로 보여줌
        self.main_url = "https://search.naver.com/search.naver"
        self.query_url = "?where=news&sm=tab_pge&query="
        self.page_url = "&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=23&mynews=0&office_type=0&"
        self.page_url += (
            "office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start="
        )

    def crawl_news_by_query(self, query: str, count: int = 200) -> List[Dict]:
        self.query = query
        parsed_query = urllib.parse.quote(query)

        if count:
            start_range = list(range(1, count, 10))
        else:
            start_range = list(range(1, 4000, 10))

        news_data = []
        for s_idx in tqdm(start_range):
            url = f"{self.main_url}{self.query_url}{parsed_query}{self.page_url}{s_idx}"

            # request and soup
            req = requests.get(url)
            soup = BeautifulSoup(req.text, "html.parser")

            # ul lists and li lists
            ul_lists = soup.find("ul", {"class": "list_news"})
            li_lists = ul_lists.findChildren("li", {"class": "bx"})
            for li in li_lists:
                # news link & title
                link = li.find("a", {"class": "news_tit"})
                href, title = link["href"], link["title"]
                
                # 언론사
                press = li.find("a", {"class": "info press"}).text
                
                # publish date
                publish_date = self._get_datetime(href, li)

                # news short_text
                short_text = li.find("a", {"class": "api_txt_lines dsc_txt_wrap"}).text
                # thumb nail if exist
                thumb = ""
                thumb_link = li.find("img", {"class": "thumb api_get"})
                if thumb_link:
                    thumb = thumb_link["src"]

                # update link_title_dict
                news_data.append(
                    {
                        "href": href,
                        "date": publish_date,
                        'press': press,
                        "title": title,
                        "short_text": short_text,
                        "thumb": thumb,
                    }
                )
            time.sleep(random.uniform(0.6, 0.9))
        return news_data

    def _get_article(self, href: str, li: bs4.element.Tag) -> str:
        links = li.find_all("a", {"class": "info"})
        naver_news_url = [link for link in links if link.text == "네이버뉴스"]
        if naver_news_url:
            naver_news_url = naver_news_url[0]
            naver_news_url = naver_news_url["href"].replace("&amp;", "&")
        else:
            naver_news_url = None
        
        if naver_news_url:
            pass

    def _get_datetime(self, href: str, li: bs4.element.Tag) -> datetime:
        try:
            a = Article(href, language="ko")
            a.download()
            a.parse()
            publish_date = a.publish_date
        except:
            publish_date = None

        if publish_date is None:
            date = li.find("span", {"class": "info"}).text
            date = date.split()[0]

            if "분" in date:
                minutes = re.sub(r"[^\d+]", "", date)
                publish_date = datetime.now() - timedelta(minutes=int(minutes))
            elif "시간" in date:
                hours = re.sub(r"[^\d+]", "", date)
                publish_date = datetime.now() - timedelta(hours=int(hours))
            elif "일" in date:
                days = re.sub(r"[^\d+]", "", date)
                publish_date = datetime.now() - timedelta(days=int(days))
            else:
                publish_date = datetime.strptime(date, "%Y.%m.%d.")
        return publish_date

## 2. Test

In [67]:
crawler = QueryNewsCrawler()

In [68]:
query = "코스피"
count = 10
news_data = crawler.crawl_news_by_query(query=query, count=count)

100%|██████████| 1/1 [00:04<00:00,  4.48s/it]


In [71]:
type(news_data[0]["date"])

datetime.datetime